In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Prepare data

In [ ]:
###
#https://github.com/tfesenko/Dog-Breed-Classifier/blob/master/dog_app.ipynb
#https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition/data?select=sample_submission.csv

In [ ]:
!ls /content/drive/MyDrive/Colab\ Notebooks/

 archive_dog_breeds.zip
'Copy of Copy of Week5 Programming assignment.ipynb'
 custom_samples
 dogs-vs-cats-redux-kernels-edition.zip
 final_project_h1.ipynb
 GoogleNews-vectors-negative300.bin
 model_transfer.pt


In [ ]:
!unzip /content/drive/MyDrive/Colab\ Notebooks/dogs-vs-cats-redux-kernels-edition.zip

Archive:  /content/drive/MyDrive/Colab Notebooks/dogs-vs-cats-redux-kernels-edition.zip
  inflating: sample_submission.csv   
  inflating: test.zip                
  inflating: train.zip               


In [ ]:
!mkdir images
!unzip train.zip -d images/

In [ ]:
!mkdir images/train/cat
!mkdir images/train/dog
!mv images/train/cat* images/train/cat
!mv images/train/dog* images/train/dog

mv: cannot move 'images/train/cat' to a subdirectory of itself, 'images/train/cat/cat'
mv: cannot move 'images/train/dog' to a subdirectory of itself, 'images/train/dog/dog'


In [ ]:
!mkdir -p images/test/cat
!mkdir -p images/test/dog
!mkdir -p images/valid/cat
!mkdir -p images/valid/dog

In [ ]:
import os

for name in os.listdir("images/train/cat")[:1200]:
  os.rename("images/train/cat/%s" % name, "images/test/cat/%s" % name)

for name in os.listdir("images/train/cat")[:1200]:
  os.rename("images/train/cat/%s" % name, "images/valid/cat/%s" % name)

for name in os.listdir("images/train/dog")[:1200]:
  os.rename("images/train/dog/%s" % name, "images/test/dog/%s" % name)

for name in os.listdir("images/train/dog")[:1200]:
  os.rename("images/train/dog/%s" % name, "images/valid/dog/%s" % name)

In [ ]:
import os
from torchvision import datasets
import torchvision.transforms as transforms
import torch

num_workers = 0
batch_size = 32
image_normalizer = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

image_transforms = {
    'train':  transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.RandomResizedCrop(224),
        transforms.ToTensor(),
        image_normalizer ]),
    'valid':  transforms.Compose([ 
        transforms.CenterCrop((224, 224)),
        transforms.ToTensor(),
        image_normalizer ]),
    'test':  transforms.Compose([
        transforms.CenterCrop((224, 224)),
        transforms.ToTensor(),
        image_normalizer ])
}

loaders_scratch = {}
images_dir = './images/'

for loader_type in ['train', 'valid', 'test']:
    dataset = datasets.ImageFolder(os.path.join(images_dir, loader_type), transform=image_transforms[loader_type])
    loader = torch.utils.data.DataLoader(dataset, shuffle=True, batch_size=batch_size, num_workers=num_workers)
    loaders_scratch[loader_type]=loader

classes = loaders_scratch['train'].dataset.classes

FileNotFoundError: ignored

# Create, train and test Resnet50

In [ ]:
import torchvision.models as models
import torch.nn as nn

model_transfer = models.resnet50(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [ ]:
# go to Edit -> Notebook setting and select GPU

use_cuda = True

for param in model_transfer.parameters():
    param.requires_grad = False
    
n_inputs = model_transfer.fc.in_features    
last_layer = nn.Linear(n_inputs, len(classes))

model_transfer.fc = last_layer
    
if use_cuda:
    model_transfer = model_transfer.cuda()

In [ ]:
# Fixes errors like 'OSError: image file is truncated (150 bytes not processed)'
# from PIL import ImageFile
# ImageFile.LOAD_TRUNCATED_IMAGES = True

import torch.optim as optim

criterion_transfer = nn.CrossEntropyLoss()

optimizer_transfer = optim.Adam(model_transfer.fc.parameters(), lr = 0.001)

In [ ]:
import time
import numpy as np

def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    valid_loss_min = np.Inf 
    
    for epoch in range(0, n_epochs):
        print("Starting training in epoch: ", epoch)
        start_time = time.time()
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        model.train()
        for (data, target) in loaders['train']:
            if use_cuda:
                data, target = data.cuda(), target.cuda()
         
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()*data.size(0)
            
        model.eval()
        for (data, target) in loaders['valid']:
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            output = model(data)
            valid_loss += criterion(output, target).item()*data.size(0)

        # calculate average losses
        train_loss = train_loss/len(loaders['train'].sampler)
        valid_loss = valid_loss/len(loaders['valid'].sampler)            
            
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
        
        ## save the model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
                valid_loss_min,
                valid_loss))
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss
            
        print('Epoch {} took {} seconds \n'.format(epoch, time.time()-start_time))    
            
    # return trained model
    return model

In [ ]:
n_epochs = 10
model_transfer = train(n_epochs, loaders_scratch, model_transfer, optimizer_transfer, criterion_transfer, use_cuda, 'model_transfer.pt')

Starting training in epoch:  0


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch: 0 	Training Loss: 0.174431 	Validation Loss: 0.060282
Validation loss decreased (inf --> 0.060282).  Saving model ...
Epoch 0 took 188.15023112297058 seconds 

Starting training in epoch:  1
Epoch: 1 	Training Loss: 0.144584 	Validation Loss: 0.055799
Validation loss decreased (0.060282 --> 0.055799).  Saving model ...
Epoch 1 took 185.60656118392944 seconds 

Starting training in epoch:  2
Epoch: 2 	Training Loss: 0.154014 	Validation Loss: 0.053383
Validation loss decreased (0.055799 --> 0.053383).  Saving model ...
Epoch 2 took 185.19200801849365 seconds 

Starting training in epoch:  3
Epoch: 3 	Training Loss: 0.141780 	Validation Loss: 0.072006
Epoch 3 took 182.9638650417328 seconds 

Starting training in epoch:  4
Epoch: 4 	Training Loss: 0.158518 	Validation Loss: 0.048516
Validation loss decreased (0.053383 --> 0.048516).  Saving model ...
Epoch 4 took 182.29166746139526 seconds 

Starting training in epoch:  5
Epoch: 5 	Training Loss: 0.154467 	Validation Loss: 0.047714

In [ ]:
!ls

drive	model_transfer.pt  sample_submission.csv  train.zip
images	sample_data	   test.zip


In [ ]:
#backup the model
!cp model_transfer.pt /content/drive/MyDrive/Colab\ Notebooks/

In [ ]:
!cp /content/drive/MyDrive/Colab\ Notebooks/model_transfer.pt .

In [ ]:
import torch
import torchvision.models as models
import torch.nn as nn

classes = ["cat", "dog"]

def load_from_checkpoint():
  checkpoint = torch.load("model_transfer.pt", map_location=torch.device('cpu'))
  # Load model here
  model = models.resnet50(pretrained=True)
  # before we load the checkpoint we need to change our final layer as we did before training
  n_inputs = model.fc.in_features    
  last_layer = nn.Linear(n_inputs, len(classes))
  model.fc = last_layer

  model.load_state_dict(checkpoint)
  model.eval()

  return model


model = None
model = load_from_checkpoint()

<All keys matched successfully>

In [ ]:
torch.save(model, 'full_model')

In [ ]:
!cp full_model /content/drive/MyDrive/Colab\ Notebooks/

In [ ]:
!cp /content/drive/MyDrive/Colab\ Notebooks/full_model .

In [ ]:
model = torch.load('full_model')
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
def test(loaders, model, criterion, use_cuda):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    model.eval()
    for batch_idx, (data, target) in enumerate(loaders['test']):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

In [ ]:
test(loaders_scratch, model_transfer, criterion_transfer, use_cuda)

Test Loss: 0.048863


Test Accuracy: 97% (2350/2400)


In [ ]:
from PIL import Image
import torch.nn.functional as F
import torchvision.transforms as transforms

# classes = ["cat", "dog"]
class_names = {e: k for e,k in enumerate(classes)}

def image_to_tensor(img):
    transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]),
        ])
    
    image_tensor = transform(img)
    return image_tensor.unsqueeze(0)

def predict_creature(img_path):
    image_tensor = image_to_tensor(Image.open(img_path)) 
    # if use_cuda:
    #     image_tensor = image_tensor.cuda()
        
    output = model(image_tensor)
    
    probabilities = F.softmax(output, dim=1).data
    
    # if use_cuda:
    #     probabilities = probabilities.cpu() # move to cpu

    return class_names[probabilities.topk(1)[1].numpy()[0][0]]
    

In [ ]:
img_path = "/content/drive/MyDrive/Colab Notebooks/custom_samples/dog_1.jpg"
a = predict_creature(img_path)

In [ ]:
a

'dog'